In [3]:
import keras
from keras.datasets import mnist
from keras.layers import Dense
from keras.models import Sequential
import pickle
import gzip
import numpy as np
from random import shuffle
from sklearn.metrics import confusion_matrix
from PIL import Image
import os
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
def One_to_N(target):
    T = [[0 for i in range(10)] for i in range(len(target))]
    for i in range(len(T)):
        j = target[i]
        T[i][j] =1
    return T
def decode(Y):
    l=[]
    for i in range(len(Y)):
        l.append(np.argmax(Y[i]))
    return l
def Get_Acc(Y,target):
    Y =np.array(Y)
    target=np.array(target)
    accuracy = 0.0
    counter = 0
    for i in range(len(Y)):
            if np.argmax(Y[i]) == target[i]:
                counter= counter+1
    accuracy = (float((counter)*100)/float(len(Y)))
    return accuracy

def Get_Acc_new(Y,target):
    Y =np.array(Y)
    target=np.array(target)
    accuracy = 0.0
    counter = 0
    for i in range(len(Y)):
            if Y[i] == target[i]:
                counter= counter+1
    accuracy = (float((counter)*100)/float(len(Y)))
    return accuracy

def gradiant(x,t,y):
    error = y -t
    return np.dot(x.T,error)/len(x)

def softmax(A):
    sum = [0 for i in range(len(A))]
    for i in range(len(A)):
        for j in range(len(A[0])):
            A[i][j] = np.exp(A[i][j])
            sum[i] = sum[i]+A[i][j]
    for i in range(len(A)):
        for j in range(len(A[0])):
            A[i][j] = A[i][j]/sum[i]
    return A

In [107]:
def logistic_regression_new(train_target,train_data,test_data,test_target_real):
    W = [[0 for i in range(len(train_data[0]))] for j in range(10)]
    W= np.array(np.matrix(W))
    L_Acc = []
    batch =500
    for i in range(50):
        for j in range(int(len(train_data)/batch)):
            train_data3 = train_data[j*batch:(j+1)*batch]
            train_target3 = train_target[j*batch:(j+1)*batch]
            A = np.dot(train_data3,W.T)
            Y = softmax(A)
            grad = gradiant(train_data3,train_target3,Y)
            Delta_W = 0.01*grad
            W_now = W - np.transpose(Delta_W)
            W = W_now
        A1 = np.dot(test_data,W.T)
        Y1 = softmax(A1)
        F= decode(Y1)
        Acc = Get_Acc(Y1,test_target_real)
        L_Acc.append(float(Acc))
    print("confusion matrix\n")
    print(confusion_matrix(F, test_target_real))
    print("\n")
    print ("Accuracy = " + str(np.around(max(L_Acc),5))) 
    return Y1

In [6]:
filename = 'mnist.pkl.gz'
f = gzip.open(filename, 'rb')
training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
f.close()

In [7]:
USPSMat  = []
USPSTar  = []
curPath  = 'USPSdata/Numerals'
savedImg = []
for j in range(0,10):
    curFolderPath = curPath + '/' + str(j)
    imgs =  os.listdir(curFolderPath)
    for img in imgs:
        curImg = curFolderPath + '/' + img
        if curImg[-3:] == 'png':
            img = Image.open(curImg,'r')
            img = img.resize((28, 28))
            savedImg = img
            imgdata = (255-np.array(img.getdata()))/255
            USPSMat.append(imgdata)
            USPSTar.append(j)

In [8]:
training_data1 = np.array(np.matrix(training_data[0]))
training_target1 = training_data[1]
validation_data1 = np.matrix(validation_data[0])
validation_target1 = validation_data[1]
testing_data1 = np.matrix(test_data[0])
testing_target1 = test_data[1]
training_target1=training_target1
print(training_data1.shape)

(50000, 784)


In [9]:
training_data2= np.array(training_data1)
validation_data2 = np.array(validation_data1)
testing_data2 = np.array(np.matrix(testing_data1))
training_target2 = np.array(One_to_N(training_target1))
testing_target2 = np.array(One_to_N(testing_target1))
validation_target2 = np.array(One_to_N(validation_target1))
USPS_Tar1 = USPSTar
USPS_Data = np.array(USPSMat)
USPS_target = np.array(One_to_N(USPSTar))
print(validation_target2[0])

[0 0 0 1 0 0 0 0 0 0]


In [10]:
MNIST_predictions = logistic_regression_new(training_target2, training_data2, testing_data2,testing_target1)

confusion matrix

[[ 956    0   11    5    2   15   16    3    9   13]
 [   0 1102    7    1    6    6    3   20   10    8]
 [   3    2  888   18    5    6    6   29   10    6]
 [   3    4   19  898    0   43    2    4   29   11]
 [   0    1   15    1  901   15   13   11    8   43]
 [   2    2    0   32    0  726   16    0   26   16]
 [   8    4   17    6   10   17  897    0   12    0]
 [   1    0   21   15    2   10    1  918   13   24]
 [   7   20   45   22    8   43    4    4  840    6]
 [   0    0    9   12   48   11    0   39   17  882]]


Accuracy = 90.08


In [11]:
USPS_predictions = logistic_regression_new(training_target2, training_data2,USPS_Data,USPS_Tar1)

confusion matrix

[[ 601  234  219  108   65  182  380  198  226   52]
 [   4  298   25    3   86   20   13  213   30  188]
 [ 375  126 1176  121   36  214  346  318  146  164]
 [  56  350  138 1259   62  184  106  450  208  470]
 [ 255  286   67   21 1028   45  105   74  127  155]
 [ 111   52   75  236  120 1031  218   78  573   84]
 [ 104   41   93   31   41  126  698   35  119   15]
 [  42  299   93   58  128   72   25  300   43  365]
 [ 147  296   90  103  292   89   75  287  444  339]
 [ 305   18   23   60  142   37   34   47   84  168]]


Accuracy = 35.01675


In [ ]:
#neural network

In [12]:
x_train = np.array(np.matrix(training_data[0]))
y_train = training_data[1]
x_test = np.matrix(test_data[0])
y_test = test_data[1]

In [13]:
num_classes=10
image_vector_size=28*28
x_train = x_train.reshape(x_train.shape[0], image_vector_size)
x_test = x_test.reshape(x_test.shape[0], image_vector_size)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
image_size = 784 
model = Sequential()
model.add(Dense(units=32, activation='sigmoid', input_shape=(image_size,)))
model.add(Dense(units=num_classes, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=124, epochs=1000, verbose=False,validation_split=.2)
loss_MNSIT,accuracy_MNSIT = model.evaluate(x_test,y_test, verbose=False)

In [90]:
print("Accuracy = "+str(accuracy_MNSIT*100))
result= model.predict(x_test)
print(confusion_matrix(decode(result), decode(y_test)))

Accuracy = 95.93
[[ 965    0    6    0    1    6    9    2    6    6]
 [   0 1121    4    1    1    2    3    6    3    6]
 [   1    2  982   12    3    0    3   16    2    1]
 [   1    1    6  968    1   15    0    9    9   10]
 [   0    0    9    1  943    1    5    1    5   25]
 [   4    1    3    8    0  838   10    0    8    7]
 [   7    5    6    2    6   12  925    0    7    0]
 [   1    2    9    8    1    2    0  981    4    7]
 [   1    3    5    8    2   10    3    1  926    3]
 [   0    0    2    2   24    6    0   12    4  944]]


In [91]:
loss_USPS,accuracy_USPS = model.evaluate(USPS_Data,USPS_target, verbose=False)
print("Accuracy = "+str(accuracy_USPS*100))
result_usps= model.predict(USPS_Data)
print(confusion_matrix(decode(result_usps), decode(USPS_target)))

Accuracy = 39.34196709760982
[[ 531   53   69   22   27   81  145   93  253   37]
 [   0  242   13    1    6   11    2   45    4   18]
 [ 148  380 1414  195   69  205  453  182  223   99]
 [ 114  225  124 1183   70  175   80  681  408  543]
 [ 171  174   26    8  976   11   45   43   72  129]
 [ 223  138  189  432  117 1315  265  136  440   49]
 [  60   39   62    4   25   44  911   12   70    7]
 [ 144  555   42   32  340   65   15  624   98  546]
 [  96  141   52  110  204   73   33  139  373  273]
 [ 513   53    8   13  166   20   51   45   59  299]]


In [111]:
classifier2 = RandomForestClassifier(n_estimators=8)
classifier2.fit(training_data1, training_target1)
result_rf = classifier2.predict(testing_data1)

In [112]:
print("Accuracy = "+str(Get_Acc_new(result_rf,testing_target1)))
print(confusion_matrix(result_rf,testing_target1))

Accuracy = 94.17
[[ 968    0   17    4    3    7   13    3    9    9]
 [   0 1123    2    2    3    1    4    8    6    5]
 [   1    2  979   18    5    4   10   27   15    3]
 [   2    2    7  940    2   40    1    7   15   13]
 [   1    0    3    1  930    5   10   10   12   29]
 [   1    2    1   19    1  808    9    2   10   11]
 [   4    4    4    0    5    6  909    0    4    2]
 [   0    2    7    9    0    3    0  951    9    7]
 [   3    0    9   12    5   11    2    0  889   10]
 [   0    0    3    5   28    7    0   20    5  920]]


In [113]:
result_rf_usps = classifier2.predict(USPS_Data)
print("Accuracy = "+str(Get_Acc_new(result_rf_usps,USPS_Tar1)))
print(confusion_matrix(result_rf_usps,USPS_Tar1))

Accuracy = 29.856492824641233
[[568  63 231 141  54 162 357  69 171  78]
 [ 88 551 148  65 243  74  86 473 181 327]
 [295 133 919 212 120 205 307 275 210 309]
 [106 141 165 927  75 278 123 225 278 283]
 [345 108  84  97 812  97 159  76 164 247]
 [192  72 151 348 192 883 294 207 607 134]
 [143  48  76  29  68  90 500  43 116  33]
 [ 97 858 192 137 323 163 124 595 102 430]
 [ 22  16  18  16  52  22  18  18 126  69]
 [144  10  15  28  61  26  32  19  45  90]]


In [19]:
classifier1 = SVC(kernel='linear', C=2, gamma = 0.05)
classifier1.fit(training_data1,training_target1)
result_svc = classifier1.predict(testing_data1)

In [94]:
print("Accuracy = "+str(Get_Acc_new(result_svc,testing_target1)))
print(confusion_matrix(result_svc,testing_target1))

Accuracy = 93.64
[[ 958    0    6    4    2   15   10    0   11    7]
 [   0 1117   11    2    1    7    3   10    6    7]
 [   5    4  960   19    9    4   11   20    7    2]
 [   1    4   13  944    0   39    1   10   24   13]
 [   1    0    3    3  944    5    5    5   10   33]
 [   3    1    1   13    0  787   13    2   22    3]
 [   8    2   12    1    5   11  912    0    8    0]
 [   1    1   10    7    1    1    1  960    9   22]
 [   1    6   14   14    2   19    2    4  869    9]
 [   2    0    2    3   18    4    0   17    8  913]]


In [99]:
result_svc_usps = classifier1.predict(USPS_Data)
print("Accuracy = "+str(Get_Acc_new(result_svc_usps,USPS_Tar1)))
print(confusion_matrix(result_svc_usps,USPS_Tar1))

Accuracy = 28.536426821341067
[[ 358   59  132   65   28   46  152   20  121   13]
 [   1  282   79   52   27   26   17   71   17   35]
 [ 493  572 1256  364  214  682  916  190  278  200]
 [ 172  265  131  884   90  249   64  715  488  579]
 [ 239  240   35   14  820   45   81   61  123  166]
 [ 316  162  224  501  213  824  250  296  648  105]
 [  69   15   61    8    8   37  450   12   83    8]
 [ 166  339   48   43  456   38   38  518   68  587]
 [  11   44   21   50   80   36    2   84  154  146]
 [ 175   22   12   19   64   17   30   33   20  161]]


In [100]:
H1 = np.array(np.matrix(decode(MNIST_predictions)))
H2 = np.array(np.matrix(decode(result)))
H3 = np.array(np.matrix(result_rf))
H4 = np.array(np.matrix(result_svc))
M= np.concatenate((H1,H2))
M1= np.concatenate((M,H3))
M2 = np.concatenate((M1,H4))
Matrix= np.array(M2.T)
Matrix1 = [[0 for i in range(10)] for j in range(10000)]

In [101]:
for i in range(10000):
    for j in Matrix[i]:
        Matrix1[i][j]= Matrix1[i][j]+1
print(np.matrix(Matrix1))

[[0 0 0 ... 4 0 0]
 [0 0 4 ... 0 0 0]
 [0 4 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [102]:
pos =[]
for i in Matrix1:
    pos.append(np.argmax(i))
print(confusion_matrix(pos,testing_target1))
print("Accuracy = "+str(Get_Acc_new(pos,testing_target1)))

[[ 971    0    9    3    2   11   10    2    9    9]
 [   0 1124    5    1    1    2    3   12    5    7]
 [   1    2  985   15    5    4    6   25    7    3]
 [   1    1    5  966    1   37    1    3   16   14]
 [   0    0    5    0  954    4    7    5    8   36]
 [   2    1    0   10    0  809   13    0   20    7]
 [   3    4    6    1    4    9  918    0    9    0]
 [   1    1    8    7    0    1    0  972    5   12]
 [   1    2    8    6    2   10    0    2  891    3]
 [   0    0    1    1   13    5    0    7    4  918]]
Accuracy = 95.08


In [103]:
H5 = np.array(np.matrix(decode(USPS_predictions)))
H6 = np.array(np.matrix(decode(result_usps)))
H7 = np.array(np.matrix(result_rf_usps))
H8 = np.array(np.matrix(result_svc_usps))
M3 = np.concatenate((H5,H6))
M4 = np.concatenate((M3,H7))
M5 = np.concatenate((M4,H8))
Matrix_usps= np.array(M5.T)
Matrix1_usps = [[0 for i in range(10)] for j in range(19999)]
for i in range(10000):
    for j in Matrix_usps[i]:
        Matrix1_usps[i][j]= Matrix1_usps[i][j]+1
print(np.matrix(Matrix1_usps))
pos_usps =[]
for i in Matrix1_usps:
    pos_usps.append(np.argmax(i))
print(confusion_matrix(pos_usps,USPS_Tar1))
print("Accuracy = "+str(Get_Acc_new(pos_usps,USPS_Tar1)))

[[0 0 0 ... 0 0 1]
 [1 0 3 ... 0 0 0]
 [3 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[ 666  128  171   83   50 1999 2000 2000 2000 2000]
 [  16  371   47   18  118    0    0    0    0    0]
 [ 387  366 1405  235  109    0    0    0    0    0]
 [  93  265   94 1264   72    1    0    0    0    0]
 [ 238  205   31   10 1052    0    0    0    0    0]
 [ 166  104  127  300  139    0    0    0    0    0]
 [  56   23   43    5   12    0    0    0    0    0]
 [  83  452   61   29  241    0    0    0    0    0]
 [  27   76   15   42  140    0    0    0    0    0]
 [ 268   10    5   14   67    0    0    0    0    0]]
Accuracy = 23.791189559477974
